In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
csv_file = pd.read_csv("/Users/a90530/Downloads/akbank/dataset/rawData.csv")
df = pd.DataFrame(csv_file)

In [ ]:
def veri_inceleme(df):
    print("\nDataframe'in ilk 5 tanesi")
    print(df.head())
    print("\n\nDataframe'in columnları")
    print(df.columns)
    print("\n\nDataframe hakkında istatistiksel bilgi")
    print(df.describe())
    print("\nColumn Sayısı:")
    print(df.shape)
    print("\n\nDataframe hakkındaki bilgileri verir")
    print(df.info())
    print("\n\nDataframe'deki boş girilen hücreleri gösterir")
    print(df.isnull().sum())
veri_inceleme(df)

In [ ]:
#Kullanılmayacak verilerin ve null değerlerin silinmesi işlemi
df = df.drop(columns=['additionalSalaryInf','jobReference'])
df = df.dropna()

In [ ]:
# Range şeklinde belirlenmiş maaş değerlerini nümerik değer haline getirme işlemi
def convert_salary_to_hourly(salary):
    if 'year' in salary:
        # Alt sınırı gerçek maaş olarak belirleme
        lower_bound_str = salary.split(' to ')[0].replace('£', '').replace(',', '').strip()
        lower_bound_str = lower_bound_str.split('per')[0]
        lower_bound = float(lower_bound_str)
        # 1 yıl içerisinde 2080 saat çalışma saati varsayımı yapıldı
        hourly_rate = lower_bound / 2080
        return round(float(hourly_rate), 2)
    elif 'hour' in salary:
        # Alt sınırı gerçek maaş olarak belirleme
        lower_bound_str = salary.split(' to ')[0].replace('£', '').replace(',', '').strip()
        lower_bound_str = lower_bound_str.split('per')[0]
        return round(float(lower_bound_str), 2)

# Apply fonksiyonu ile salary sütununa "extract_salary" fonksiyonunu uyguluyoruz
df['salary_numeric'] = df['salary'].apply(convert_salary_to_hourly)
df.dropna(subset=['salary_numeric'], inplace=True)

In [ ]:
import sys
sys.path.append('/Users/a90530/Downloads/akbank/modules')
from local_outlier import LocalOutlierDetection

#Aykırı değerlerin değişmesi
outlier_detection = LocalOutlierDetection()
outlier_detection.set_neighbors(20)
df['salary_numeric'] = outlier_detection.remove_outliers(df, 'salary_numeric')['salary_numeric']

In [ ]:
# Sektörlere göre ortalama maaş dağılımı
average_salary_by_category = df.groupby("category")["salary_numeric"].mean().reset_index()

plt.figure(figsize=(10, 6))
plt.bar(average_salary_by_category["category"], average_salary_by_category["salary_numeric"], color='skyblue')
plt.xlabel('Sektör')
plt.ylabel('Ortalama Maaş (£)')
plt.title('Her Sektör İçin Ortalama Maaş Grafiği')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()
